In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
%load_ext google.cloud.bigquery
from google.cloud import bigquery

client=bigquery.Client()
project_id = "som-nero-phi-jonc101"

/Users/sandychen/Desktop/Healthrex_workspace/scripts/Blood_Culture_Stewardship/blood_culture_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/sandychen/Desktop/Healthrex_workspace/scripts/Blood_Culture_Stewardship/blood_culture_env/lib/python3.9/site-packages/google/cloud/bigquery/__init__.py:237: FutureWarning: %load_ext google.cloud.bigquery is deprecated. Install bigquery-magics package and use `%load_ext bigquery_magics`, instead.
  warnings.warn(


In [ ]:
# # Test_set_df_note_ORIGINAL = pd.read_csv("../data/Test_set_df_note_ORIGINAL.csv")
# table_id_all_features_imputed_test= f"{project_id}.blood_culture_stewardship_sandy_2024.final_base_features_imputed_test"
# Test_set_df_note_ORIGINAL = client.query(f"SELECT * FROM `{table_id_all_features_imputed_test}`").to_dataframe()

In [3]:
# demo
table_id_final_base_demo =  f"{project_id}.blood_culture_stewardship_sandy_2024.enriched_label_filtered_adult_only_analysis_cohort"

final_base_demo = client.query(f"SELECT * FROM `{table_id_final_base_demo}`").to_dataframe()
# labs
table_id_final_base_labs_lda_ua = f"{project_id}.blood_culture_stewardship_sandy_2024.final_base_labs_without_cpr_or_pct_lda_encouter_level_adult"
final_base_labs_ua_lda= client.query(f"SELECT * FROM `{table_id_final_base_labs_lda_ua}`").to_dataframe()
# vitals
table_id_final_base_vitals =f"{project_id}.blood_culture_stewardship_sandy_2024.final_base_vitals_adult_encounter_level"
final_base_vitals= client.query(f"SELECT * FROM `{table_id_final_base_vitals}`").to_dataframe()

table_id_final_labs_cr_only = f"{project_id}.blood_culture_stewardship_sandy_2024.final_base_labs_cr_only_encounter_level_adult"
final_labs_cr_only = client.query(f"SELECT * FROM `{table_id_final_labs_cr_only}`").to_dataframe()
# Alternative: Using functools.reduce to merge multiple dataframes at once
from functools import reduce

# Define merge keys
key_cols = ["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded", "blood_culture_order_datetime_utc"]

# List of dataframes to merge
dfs_to_merge = [final_base_demo, final_base_labs_ua_lda, final_base_vitals, final_labs_cr_only]

# Merge all dataframes
all_features_df = reduce(
    lambda left, right: pd.merge(left, right, on=key_cols, how="inner"),
    dfs_to_merge
)

print(f"Shape of merged features: {all_features_df.shape}")
print(f"Number of rows: {len(all_features_df)}")
all_features_df.head()


all_features_df["label"] = all_features_df["final_label"] == "positive"
test_data = all_features_df[all_features_df['order_year'] >= 2024]

Shape of merged features: (101812, 112)
Number of rows: 101812


In [4]:
Test_set_df_note_ORIGINAL = test_data.copy()

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

# ---------- 1) Add the six criteria ----------
def add_criteria(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # --- SIRS Criteria ---
    # df["sirsTemp"]      = np.where((df["temp_min_c"] < 96.8) | (df["temp_max_c"] > 100.4), 1, 0)
    df["sirsTemp"]      = np.where((df["temp_min_c"] < 36) | (df["temp_max_c"] > 38), 1, 0)
    
    df["sirsHeartRate"] = np.where(df["hr_max"] > 90, 1, 0)
    df["sirsRespRate"]  = np.where(df["resp_max"] > 20, 1, 0)
    df["sirsWBC"]       = np.where((df["min_wbc"] < 4.0) | (df["max_wbc"] > 12.0), 1, 0)

    # --- Hypotension / perfusion ---
    df["lowBP"]       = np.where(df["sysbp_min"] < 90, 1, 0)
    df["highLactate"] = np.where(df["max_lactate"] > 2.0, 1, 0)

    # --- Shapiro criteria ---
    df["shapiroHighCr"]          = np.where(df["max_cr"] > 2.0, 1, 0)
    df["shapiroHighTemp"]        = np.where(df["temp_max_c"] > 101.0, 1, 0)
    df["shapiroHighNeutrophils"] = np.where(df["max_neutrophils"] > 80, 1, 0)
    df["shapiroHighWBC"]         = np.where(df["max_wbc"] > 18, 1, 0)
    df["shapiroLowPlt"]          = np.where(df["min_plt"] < 150, 1, 0)
    df["shapiroHighAge"]         = np.where(df["age"] > 65, 1, 0)

    # --- Composite SIRS score ---
    df["sirsScore"] = df[["sirsTemp","sirsHeartRate","sirsRespRate","sirsWBC"]].sum(axis=1)

    # --- The six definitions ---
    df["SIRS (2+ criteria)"]          = (df["sirsScore"] >= 2).astype(int)
    df["SIRS (1+ criteria)"]          = (df["sirsScore"] >= 1).astype(int)
    df["sirs2orLowBPorHighLactate"]   = ((df["sirsScore"] >= 2) | ((df["lowBP"] + df["highLactate"]) >= 1)).astype(int)
    df["anySIRSorLowBPorHighLactate"] = ((df["sirsScore"] + df["lowBP"] + df["highLactate"]) >= 1).astype(int)
    df["shapiroCriteria2orMore"]      = (df[[
        "shapiroHighTemp","shapiroHighAge","lowBP","shapiroHighWBC",
        "shapiroHighNeutrophils","shapiroLowPlt","shapiroHighCr"
    ]].sum(axis=1) >= 2).astype(int)
    df["shapiroCriteria1orMore"]      = (df[[
        "shapiroHighTemp","shapiroHighAge","lowBP","shapiroHighWBC",
        "shapiroHighNeutrophils","shapiroLowPlt","shapiroHighCr"
    ]].sum(axis=1) >= 1).astype(int)

    return df


# ---------- 2) Metric helper ----------
def bin_metrics(y_true, y_pred):
    mask = ~(pd.isna(y_true) | pd.isna(y_pred))
    y_t = np.asarray(y_true[mask]).astype(int)
    y_p = np.asarray(y_pred[mask]).astype(int)

    if y_t.size == 0:
        return dict(tn=np.nan, fp=np.nan, fn=np.nan, tp=np.nan,
                    Prevalence=np.nan, Sensitivity=np.nan, Specificity=np.nan,
                    PPV=np.nan, NPV=np.nan)

    tn, fp, fn, tp = confusion_matrix(y_t, y_p, labels=[0,1]).ravel()

    sens = tp / (tp + fn) if (tp + fn) > 0 else np.nan
    spec = tn / (tn + fp) if (tn + fp) > 0 else np.nan
    ppv  = tp / (tp + fp) if (tp + fp) > 0 else np.nan
    npv  = tn / (tn + fn) if (tn + fn) > 0 else np.nan
    prev = (tp + fn) / (tn + fp + fn + tp) if (tn + fp + fn + tp) > 0 else np.nan

    return dict(tn=tn, fp=fp, fn=fn, tp=tp,
                Prevalence=prev, Sensitivity=sens, Specificity=spec, PPV=ppv, NPV=npv)


# ---------- 3) Evaluate exactly the six ----------
def evaluate_six(df: pd.DataFrame, outcome_col: str = "Label") -> pd.DataFrame:
    rules = [
        "SIRS (2+ criteria)",
        "sirs2orLowBPorHighLactate",
        "SIRS (1+ criteria)",
        "anySIRSorLowBPorHighLactate",
        "shapiroCriteria2orMore",
        "shapiroCriteria1orMore",
    ]

    rows = []
    for rule in rules:
        m = bin_metrics(df[outcome_col], df[rule])
        m["Criteria"] = rule
        rows.append(m)

    out = (pd.DataFrame(rows)
             .loc[:, ["Criteria","tn","fp","fn","tp",
                      "Prevalence","Sensitivity","Specificity","PPV","NPV"]])
    # round for readability
    for k in ["Prevalence","Sensitivity","Specificity","PPV","NPV"]:
        out[k] = out[k].round(4)
    return out



In [7]:
df = add_criteria(Test_set_df_note_ORIGINAL)
results = evaluate_six(df, outcome_col="label")
print(results.to_string(index=False))

                   Criteria   tn    fp  fn   tp  Prevalence  Sensitivity  Specificity    PPV    NPV
         SIRS (2+ criteria) 5796  8285 210  853      0.0702       0.8024       0.4116 0.0933 0.9650
  sirs2orLowBPorHighLactate 4702  9379 146  917      0.0702       0.8627       0.3339 0.0891 0.9699
         SIRS (1+ criteria) 2043 12038  33 1030      0.0702       0.9690       0.1451 0.0788 0.9841
anySIRSorLowBPorHighLactate 1772 12309  26 1037      0.0702       0.9755       0.1258 0.0777 0.9855
     shapiroCriteria2orMore 8191  5890 317  746      0.0702       0.7018       0.5817 0.1124 0.9627
     shapiroCriteria1orMore 3265 10816  67  996      0.0702       0.9370       0.2319 0.0843 0.9799
